In [61]:
import numpy as np

In [62]:
def calculate_distance(city1, city2, distance_dict):
    return distance_dict[(city1, city2)]

def compute_distance(route, distance_dict, cluster_dict=None):
    full_route = []
    for item in route:
        if cluster_dict and item in cluster_dict:
            full_route.extend(cluster_dict[item])
        else:
            full_route.append(item)

    dist = 0.0
    for i in range(len(full_route)):
        current_city = full_route[i]
        next_city = full_route[(i+1) % len(full_route)]
        dist += calculate_distance(current_city, next_city, distance_dict)
    return dist


# Step 1: Clustering

In [63]:
# Number of cities
np.random.seed(9)

num_cities = 20
level_0_iterations = 2
level_1_iterations = 2
level_2_iterations = 2
max_distance = 5

# Creating cities with random distances between 1 and 5
all_cities = [f"City {i}" for i in range(1, num_cities+1)]

distances = {}
for i in range(num_cities):
    for j in range(i+1, num_cities):
        dist = np.random.randint(1, max_distance-1)
        cityA = all_cities[i]
        cityB = all_cities[j]
        distances[(cityA, cityB)] = dist
        distances[(cityB, cityA)] = dist

In [64]:
print("\nLevel 0")

# Shuffling cities and calculating distance for the initial order
city_names = list(all_cities)
np.random.shuffle(city_names)
print(city_names)

initial_distance = compute_distance(city_names, distances)
print(f"Initial distance: {initial_distance:.2f}")


Level 0
['City 11', 'City 1', 'City 9', 'City 4', 'City 17', 'City 20', 'City 18', 'City 15', 'City 8', 'City 6', 'City 19', 'City 10', 'City 5', 'City 13', 'City 3', 'City 14', 'City 2', 'City 16', 'City 7', 'City 12']
Initial distance: 44.00


In [65]:
# Level 1. Creating clusters from random pairs of cities
city_list = list(city_names)
np.random.shuffle(city_list)

clusters_level_1 = {}

# Level 1 clusering. Popping two random elements from the list, and appending to level 1 cluster
for i in range(int(num_cities/2)):
    city_a = city_list.pop()
    city_b = city_list.pop()
    clusters_level_1[f"Cluster {i+1}"] = (city_a, city_b)

# Printing level 1 clusters
print("\nLevel 1")
for cluster, cities in clusters_level_1.items():
    print(f"{cluster}: {cities}")


Level 1
Cluster 1: ('City 13', 'City 9')
Cluster 2: ('City 6', 'City 8')
Cluster 3: ('City 12', 'City 7')
Cluster 4: ('City 2', 'City 1')
Cluster 5: ('City 11', 'City 15')
Cluster 6: ('City 16', 'City 3')
Cluster 7: ('City 5', 'City 14')
Cluster 8: ('City 4', 'City 17')
Cluster 9: ('City 18', 'City 19')
Cluster 10: ('City 20', 'City 10')


In [66]:
# Level 2. Creating clusters of random pair of clusters
cluster_list = list(clusters_level_1.keys())
np.random.shuffle(cluster_list)

clusters_level_2 = {}
l2_to_l1_map = {}

# Level 2 clustering. Popping two random clusters from the list and appending to level 2 cluster
for i in range(int(num_cities/4)):
    cluster_a_name = cluster_list.pop()
    cluster_b_name = cluster_list.pop()

    cluster_a_tuple = clusters_level_1[cluster_a_name]
    cluster_b_tuple = clusters_level_1[cluster_b_name]

    l2_name = f"Cluster {i+1}"

    clusters_level_2[l2_name] = cluster_a_tuple + cluster_b_tuple
    l2_to_l1_map[l2_name] = (cluster_a_name, cluster_b_name)

# Printing level 2 clusters
print("\nLevel 2")
for cluster, cities_4 in clusters_level_2.items():
    print(f"{cluster}: {cities_4}")


Level 2
Cluster 1: ('City 13', 'City 9', 'City 20', 'City 10')
Cluster 2: ('City 6', 'City 8', 'City 11', 'City 15')
Cluster 3: ('City 18', 'City 19', 'City 2', 'City 1')
Cluster 4: ('City 5', 'City 14', 'City 12', 'City 7')
Cluster 5: ('City 16', 'City 3', 'City 4', 'City 17')


# Step 2: Optimizing at level 2

In [67]:
print("\nStep 2, two iteration search algorithm at level 2\n")

# Shuffling the order of the cluster and calculating distance
cluster_order = list(clusters_level_2.keys())
np.random.shuffle(cluster_order)
current_distance = compute_distance(cluster_order, distances, clusters_level_2)

# Printing initial solution and cost
print("Initial cluster order:", cluster_order)
initial_solution_level2 = [{clusters_level_2[ck]} for ck in cluster_order]
print(f"Initial solution: {initial_solution_level2}")
print(f"Initial distance: {current_distance:.2f}")

# Two iterations. Choosing 2 random indices, swapping the clusters at these positions are recalculate the distance. Keep if shorter, or revert if longer.
for iteration in range(level_2_iterations):
    i, j = np.random.choice(int(num_cities/4), size=2, replace=False)
    print(f"\nIteration {iteration+1}:")
    print(f"Random indices: {i}, {j}")

    # Swapping based on the random indices
    swapped_order = cluster_order[:]
    swapped_order[i], swapped_order[j] = swapped_order[j], swapped_order[i]
    print("Swapped order:", swapped_order)

    # Computing the distance for the swapped order
    new_distance = compute_distance(swapped_order, distances, clusters_level_2)
    print(f"Swapped distance: {new_distance:.2f}")

    # Keep new order if it is shorter than original
    if new_distance < current_distance:
        print("Keeping swapped order")
        cluster_order = swapped_order
        current_distance = new_distance
    else:
        print("Reverting to previous order")

# Printing final solution for given level
print("\nFinal cluster order:", cluster_order)
final_solution_level2 = [{clusters_level_2[ck]} for ck in cluster_order]
print(f"Final solution at level 2: {final_solution_level2}")
print(f"Final distance: {current_distance:.2f}")

final_order_l2 = cluster_order[:]


Step 2, two iteration search algorithm at level 2

Initial cluster order: ['Cluster 4', 'Cluster 5', 'Cluster 3', 'Cluster 2', 'Cluster 1']
Initial solution: [{('City 5', 'City 14', 'City 12', 'City 7')}, {('City 16', 'City 3', 'City 4', 'City 17')}, {('City 18', 'City 19', 'City 2', 'City 1')}, {('City 6', 'City 8', 'City 11', 'City 15')}, {('City 13', 'City 9', 'City 20', 'City 10')}]
Initial distance: 46.00

Iteration 1:
Random indices: 3, 4
Swapped order: ['Cluster 4', 'Cluster 5', 'Cluster 3', 'Cluster 1', 'Cluster 2']
Swapped distance: 48.00
Reverting to previous order

Iteration 2:
Random indices: 2, 1
Swapped order: ['Cluster 4', 'Cluster 3', 'Cluster 5', 'Cluster 2', 'Cluster 1']
Swapped distance: 43.00
Keeping swapped order

Final cluster order: ['Cluster 4', 'Cluster 3', 'Cluster 5', 'Cluster 2', 'Cluster 1']
Final solution at level 2: [{('City 5', 'City 14', 'City 12', 'City 7')}, {('City 18', 'City 19', 'City 2', 'City 1')}, {('City 16', 'City 3', 'City 4', 'City 17')}, {

# Step 3: Optimizing at level 1

In [68]:
print("\nStep 3, two iteration search algorithm at level 1\n")

# Expanding from level 2 to level 1
level1_order = []
for l2_name in final_order_l2:
    l1_a, l1_b = l2_to_l1_map[l2_name]
    level1_order.append(l1_a)
    level1_order.append(l1_b)

# Printing initial solution and distance
print("Expanded cluster order:", level1_order)
initial_solution_level1 = [{clusters_level_1[name]} for name in level1_order]
print(f"Initial solution: {initial_solution_level1}")
current_distance_l1 = compute_distance(level1_order, distances, clusters_level_1)
print(f"Initial distance: {current_distance_l1:.2f}")

# Two iterations. Choosing 2 random indices, swapping the clusters at these positions are recalculate the distance. Keep if shorter, or revert if longer.
for iteration in range(level_1_iterations):
    i, j = np.random.choice(int(num_cities/2), size=2, replace=False)
    print(f"\nIteration {iteration+1}:")
    print(f"Random indices: {i}, {j}")

    # Swapping based on the random indices
    swapped_order = level1_order[:]
    swapped_order[i], swapped_order[j] = swapped_order[j], swapped_order[i]
    print("Swapped order:", swapped_order)

    # Computing the distance for the swapped order
    new_distance = compute_distance(swapped_order, distances, clusters_level_1)
    print(f"Swapped distance: {new_distance:.2f}")

    # Keep new order if it is shorter than original
    if new_distance < current_distance_l1:
        print("Keeping swapped order")
        level1_order = swapped_order
        current_distance_l1 = new_distance
    else:
        print("Reverting to previous order")

# Printing final solution for given level
print("\nFinal cluster order:", level1_order)
final_solution_level1 = [{clusters_level_1[ck]} for ck in level1_order]
print(f"Final solution at level 1: {final_solution_level1}")
print(f"Final distance: {current_distance_l1:.2f}")


Step 3, two iteration search algorithm at level 1

Expanded cluster order: ['Cluster 7', 'Cluster 3', 'Cluster 9', 'Cluster 4', 'Cluster 6', 'Cluster 8', 'Cluster 2', 'Cluster 5', 'Cluster 1', 'Cluster 10']
Initial solution: [{('City 5', 'City 14')}, {('City 12', 'City 7')}, {('City 18', 'City 19')}, {('City 2', 'City 1')}, {('City 16', 'City 3')}, {('City 4', 'City 17')}, {('City 6', 'City 8')}, {('City 11', 'City 15')}, {('City 13', 'City 9')}, {('City 20', 'City 10')}]
Initial distance: 43.00

Iteration 1:
Random indices: 1, 0
Swapped order: ['Cluster 3', 'Cluster 7', 'Cluster 9', 'Cluster 4', 'Cluster 6', 'Cluster 8', 'Cluster 2', 'Cluster 5', 'Cluster 1', 'Cluster 10']
Swapped distance: 45.00
Reverting to previous order

Iteration 2:
Random indices: 2, 1
Swapped order: ['Cluster 7', 'Cluster 9', 'Cluster 3', 'Cluster 4', 'Cluster 6', 'Cluster 8', 'Cluster 2', 'Cluster 5', 'Cluster 1', 'Cluster 10']
Swapped distance: 44.00
Reverting to previous order

Final cluster order: ['Cluste

# Step 4: Optimizing at level 0

In [69]:
print("\nStep 4: two-iteration local search algorithm at the original level\n")

# Expanding from level 1 to level 0
final_city_route = []
for ck in level1_order:
    city_pair = clusters_level_1[ck]
    final_city_route.extend(city_pair)

# Printing initial solution for level 0
print("Extended city route")
print(final_city_route)
current_distance_l0 = compute_distance(final_city_route, distances)
print(f"Initial distance: {current_distance_l0:.2f}")

best_route = final_city_route[:]

# Two iterations. Choosing 2 random indices, swapping the cities at these positions are recalculate the distance. Keep if shorter, or revert if longer.
for iteration in range(level_0_iterations):
    i, j = np.random.choice(num_cities, size=2, replace=False)
    print(f"\nIteration {iteration+1}:")
    print(f"Random indices: {i}, {j}")

    # Swapping based on the random indices
    swapped = best_route[:]
    swapped[i], swapped[j] = swapped[j], swapped[i]

    # Computing the distance for the swapped order
    new_dist = compute_distance(swapped, distances)
    print(f"Swapped distance: {new_dist:.2f}")

    # Keep new order if it is shorter than original 
    if new_dist < current_distance_l0:
        print("Keeping swapped order")
        best_route = swapped
        current_distance_l0 = new_dist
    else:
        print("Reverting to previous order")

# Printing final best route
print("\nFinal city route order:")
print(best_route)
print(f"Final distance: {current_distance_l0:.2f}")


Step 4: two-iteration local search algorithm at the original level

Extended city route
['City 5', 'City 14', 'City 12', 'City 7', 'City 18', 'City 19', 'City 2', 'City 1', 'City 16', 'City 3', 'City 4', 'City 17', 'City 6', 'City 8', 'City 11', 'City 15', 'City 13', 'City 9', 'City 20', 'City 10']
Initial distance: 43.00

Iteration 1:
Random indices: 17, 16
Swapped distance: 42.00
Keeping swapped order

Iteration 2:
Random indices: 19, 15
Swapped distance: 46.00
Reverting to previous order

Final city route order:
['City 5', 'City 14', 'City 12', 'City 7', 'City 18', 'City 19', 'City 2', 'City 1', 'City 16', 'City 3', 'City 4', 'City 17', 'City 6', 'City 8', 'City 11', 'City 15', 'City 9', 'City 13', 'City 20', 'City 10']
Final distance: 42.00
